In [24]:
from datasets import load_dataset, ClassLabel
ds = load_dataset("tianharjuno/twitter-parse", cache_dir="/data/cache/")
train_ds = ds["train"]
test_ds = ds["test"]
# Split into 80% train, 20% temp (for eval + test)
train_ds_split= train_ds.train_test_split(test_size=0.2, seed=42)

train_ds = train_ds_split["train"]
eval_ds = train_ds_split["test"]

print(train_ds, eval_ds, test_ds)

print(f"Train: {len(train_ds)}, Eval: {len(eval_ds)}, Test: {len(test_ds)}")

class_labels = ClassLabel(names=list(set(train_ds["relevant"])))

train_ds = train_ds.cast_column("relevant", class_labels)
eval_ds = eval_ds.cast_column("relevant", class_labels)
test_ds = test_ds.cast_column("relevant", class_labels)

cleaned_train_ds = train_ds.select_columns(["content", "relevant"])
cleaned_test_ds = test_ds.select_columns(["content", "relevant"])
cleaned_eval_ds = eval_ds.select_columns(["content", "relevant"])
cleaned_train_ds = cleaned_train_ds.rename_column("relevant", "label")
cleaned_eval_ds = cleaned_eval_ds.rename_column("relevant", "label")
cleaned_test_ds = cleaned_test_ds.rename_column("relevant", "label")

cleaned_train_ds = cleaned_train_ds.rename_column("content", "text")
cleaned_eval_ds = cleaned_eval_ds.rename_column("content", "text")
cleaned_test_ds = cleaned_test_ds.rename_column("content", "text")

Generating source_stage_1 split:   0%|          | 0/201583 [00:00<?, ? examples/s]

Generating source_stage_2 split:   0%|          | 0/247820 [00:00<?, ? examples/s]

Generating cleaned split:   0%|          | 0/195952 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/19999 [00:00<?, ? examples/s]

Dataset({
    features: ['tweet_id', 'time', 'author', 'content', 'comment_count', 'repost_count', 'like_count', 'view_count', 'relevant', 'sentiment'],
    num_rows: 15999
}) Dataset({
    features: ['tweet_id', 'time', 'author', 'content', 'comment_count', 'repost_count', 'like_count', 'view_count', 'relevant', 'sentiment'],
    num_rows: 4000
}) Dataset({
    features: ['tweet_id', 'time', 'author', 'content', 'comment_count', 'repost_count', 'like_count', 'view_count', 'relevant', 'sentiment'],
    num_rows: 5000
})
Train: 15999, Eval: 4000, Test: 5000


Casting the dataset:   0%|          | 0/15999 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [25]:
print(cleaned_train_ds, cleaned_eval_ds, cleaned_test_ds)

Dataset({
    features: ['text', 'label'],
    num_rows: 15999
}) Dataset({
    features: ['text', 'label'],
    num_rows: 4000
}) Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})


In [26]:
from collections import Counter

# Extract all labels
labels = cleaned_train_ds["label"]

# Count occurrences
label_counts = Counter(labels)

num_false = label_counts.get(0, 0)  # count of label 0 / False
num_true = label_counts.get(1, 0)   # count of label 1 / True

print(f"False: {num_false}, True: {num_true}")

print(label_counts)

False: 6384, True: 9615
Counter({1: 9615, 0: 6384})


In [27]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

device = torch.device("cuda")

# Load model & tokenizer
model = AutoModelForSequenceClassification.from_pretrained(
    "indolem/indobertweet-base-uncased",
    cache_dir="/data/cache/"
)
tokenizer = AutoTokenizer.from_pretrained(
    "indolem/indobertweet-base-uncased",
    cache_dir="/data/cache/"
)

# Move model to device
model.to(device)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31923, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [28]:
def tokenize(key):
    def callback(row):
        return tokenizer(
            row[key],
            truncation=True,
            padding="max_length",
            max_length=128,
        )
    return callback

In [29]:
tokenizer_callback = tokenize("text")
encoded_train_ds = cleaned_train_ds.map(tokenizer_callback, batch_size=256, batched=True)
encoded_eval_ds = cleaned_eval_ds.map(tokenizer_callback, batch_size=256, batched=True)
encoded_test_ds = cleaned_test_ds.map(tokenizer_callback, batch_size=256, batched=True)

Map:   0%|          | 0/15999 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [30]:
encoded_train_ds.set_format("torch", columns=["label", "input_ids", "attention_mask"])
encoded_eval_ds.set_format("torch", columns=["label", "input_ids", "attention_mask"])
encoded_test_ds.set_format("torch", columns=["label", "input_ids", "attention_mask"])


In [31]:
from transformers import Trainer
from torch import nn

class WeightedLossTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        weights = self.class_weights.to(logits.device)
        loss_fct = nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), 
                        labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [32]:
from transformers.training_args import TrainingArguments
from transformers.trainer_callback import EarlyStoppingCallback
from transformers.trainer import Trainer
from transformers.data.data_collator import default_data_collator
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np

def compute_metrics(class_names):
    num_classes = len(class_names)
    def callback(eval_pred):
        logits, labels = eval_pred
        if isinstance(logits, torch.Tensor):
            logits = logits.detach().cpu().numpy()
        if isinstance(labels, torch.Tensor):
            labels = labels.detach().cpu().numpy()
        preds = np.argmax(logits, axis=1)
        macro_p, macro_r, macro_f1, _ = precision_recall_fscore_support(
            labels, preds, average="macro", zero_division=0
        )
        acc = accuracy_score(labels, preds)
        p_cls, r_cls, f1_cls, support_cls = precision_recall_fscore_support(
            labels, 
            preds, 
            average=None,
            zero_division=0,
            labels=list(range(num_classes))
        )
        metrics = {
            "accuracy": acc,
            "macro_f1": macro_f1,
            "macro_precision": macro_p,
            "macro_recall": macro_r,
        }
        for idx, name in enumerate(class_names):
            metrics[f"{name}_precision"] = p_cls[idx] #type: ignore
            metrics[f"{name}_recall"]    = r_cls[idx]  #type: ignore
            metrics[f"{name}_f1"]        = f1_cls[idx]  #type: ignore
            metrics[f"{name}_support"]   = int(support_cls[idx])  #type: ignore
        return metrics
    return callback


training_args = TrainingArguments(
    output_dir="/data",
    overwrite_output_dir=True,
    eval_strategy="epoch",     # evaluate at the end of each epoch
    save_strategy="epoch",           # save checkpoint at the end of each epoch
    learning_rate=1e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=256,
    num_train_epochs=10,
    weight_decay=0.05,
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_macro_f1",
    greater_is_better=True,
    warmup_ratio=0.01,
    fp16=True,
)

def train_model(
    model,
    train_dataset,
    eval_dataset,
    test_dataset,
    input_class_names,
    class_weights
):
    compute_callback = compute_metrics(input_class_names)
    trainer = WeightedLossTrainer(
        model = model,
        args = training_args,
        train_dataset = train_dataset,
        eval_dataset=eval_dataset,
        data_collator=default_data_collator,
        compute_metrics=compute_callback,
        class_weights=class_weights
    )
    print("Starting training...")
    trainer.train()
    print("Training complete. Evaluating...")
    return trainer.evaluate(eval_dataset = test_dataset), trainer.model

In [33]:
import os
num_false = label_counts.get(False, 0)
num_true = label_counts.get(True, 0)
total = num_false + num_true

my_weights = torch.tensor([num_true / total,  # weight for class 0 (False)
                           num_false / total], # weight for class 1 (True)
                           dtype=torch.float)
results, trained_model = train_model(model, encoded_train_ds, encoded_eval_ds, encoded_test_ds, class_labels.names, my_weights)

Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1,Macro Precision,Macro Recall,False Precision,False Recall,False F1,False Support,True Precision,True Recall,True F1,True Support
1,0.383600,0.258017,0.907500,0.904438,0.902179,0.907270,0.869332,0.906095,0.887333,1608,0.935026,0.908445,0.921544,2392
2,0.240000,0.242157,0.911000,0.907881,0.906160,0.909890,0.878019,0.904229,0.890931,1608,0.934300,0.915552,0.924831,2392
3,0.211300,0.239839,0.911500,0.908485,0.906470,0.910920,0.876351,0.907960,0.891875,1608,0.936590,0.913880,0.925095,2392
4,0.188500,0.247108,0.909500,0.905788,0.906213,0.905375,0.889862,0.884328,0.887087,1608,0.922565,0.926421,0.924489,2392
5,0.170200,0.245701,0.908750,0.906436,0.902793,0.913003,0.852524,0.934701,0.891724,1608,0.953062,0.891304,0.921149,2392
6,0.149900,0.253736,0.909500,0.906346,0.904572,0.908432,0.875754,0.902985,0.889161,1608,0.933390,0.913880,0.923532,2392
7,0.131900,0.262532,0.909250,0.906237,0.903993,0.909039,0.871642,0.907960,0.889430,1608,0.936344,0.910117,0.923044,2392
8,0.121800,0.268980,0.909250,0.906255,0.903959,0.909141,0.871199,0.908582,0.889498,1608,0.936720,0.909699,0.923012,2392
9,0.113100,0.273791,0.907500,0.904646,0.901841,0.908493,0.864118,0.913557,0.888150,1608,0.939565,0.903428,0.921142,2392
10,0.109600,0.276891,0.907500,0.904420,0.902212,0.907168,0.869773,0.905473,0.887264,1608,0.934652,0.908863,0.921577,2392


Training complete. Evaluating...


In [39]:
results

{'eval_loss': 0.24182546138763428,
 'eval_accuracy': 0.9082,
 'eval_macro_f1': 0.9037855219555779,
 'eval_macro_precision': 0.9010565164892763,
 'eval_macro_recall': 0.9070816456317661,
 'eval_False_precision': 0.8649052841475573,
 'eval_False_recall': 0.9022360894435777,
 'eval_False_f1': 0.8831763807584627,
 'eval_False_support': 1923,
 'eval_True_precision': 0.9372077488309953,
 'eval_True_recall': 0.9119272018199545,
 'eval_True_f1': 0.9243946631526931,
 'eval_True_support': 3077,
 'eval_runtime': 2.0899,
 'eval_samples_per_second': 2392.416,
 'eval_steps_per_second': 9.57,
 'epoch': 10.0}

In [41]:
import pandas as pd
import numpy as np
import torch
from transformers import Trainer, TrainingArguments
from IPython.display import display, HTML

# We don't need a full training setup, just a place to output predictions.
# This creates a dummy TrainingArguments object.
training_args = TrainingArguments(
    output_dir="/data",
    per_device_eval_batch_size=64, # Use a large batch size for fast eval
)

# Initialize the Trainer
trainer = Trainer(

    model=trained_model,
    args=training_args,
)

print("Running predictions on test_ds...")

# --- 2. Run Predictions ---
# The .predict() method runs inference on test_ds and returns a PredictionOutput object
prediction_output = trainer.predict(encoded_test_ds)

print("Predictions complete.")

# The predictions are logits (raw model scores). We need the class index (0 or 1).
predicted_labels = np.argmax(prediction_output.predictions, axis=-1)

# The true labels are also in the output, just to be safe
true_labels = prediction_output.label_ids

# --- 3. Create DataFrame for Review ---

# Get the original text from the dataset
# This is why your test_ds *must* have the 'text' column
try:
    original_texts = encoded_test_ds['text']
except KeyError:
    print("="*50)
    print("ERROR: Your 'test_ds' does not have a 'text' column.")
    print("Please reload your dataset without removing the 'text' column.")
    print("="*50)
    # Stop execution if the text column is missing
    raise

# Create the main DataFrame
df = pd.DataFrame({
    'text': original_texts,
    'true_label': true_labels,
    'predicted_label': predicted_labels
})

# --- 4. Isolate and Categorize Errors ---

# Filter to get only the rows where the model was wrong
errors_df = df[df['true_label'] != df['predicted_label']].copy()

# Add a new column to categorize the error type
def get_error_type(row):
    if row['true_label'] == 0 and row['predicted_label'] == 1:
        # Model predicted 1 (Relevant), but it was 0 (Not Relevant)
        return 'False Positive (FP)'
    elif row['true_label'] == 1 and row['predicted_label'] == 0:
        # Model predicted 0 (Not Relevant), but it was 1 (Relevant)
        return 'False Negative (FN)'

errors_df['error_type'] = errors_df.apply(get_error_type, axis=1)

print(f"\nFound {len(errors_df)} misclassified samples out of {len(df)} total.")

# --- 5. Display the Wrong Answers for Review ---

# Set pandas display options for better text viewing in the notebook
pd.set_option('display.max_colwidth', 300) # Show more text
pd.set_option('display.max_rows', 100)     # Show more rows

print("\n--- 🔴 FALSE POSITIVES (Model said 'Relevant', but was 'Not Relevant') ---")
display(errors_df[errors_df['error_type'] == 'False Positive (FP)'])

print("\n--- 🔵 FALSE NEGATIVES (Model said 'Not Relevant', but was 'Relevant') ---")
display(errors_df[errors_df['error_type'] == 'False Negative (FN)'])

Running predictions on test_ds...


Predictions complete.

Found 459 misclassified samples out of 5000 total.

--- 🔴 FALSE POSITIVES (Model said 'Relevant', but was 'Not Relevant') ---


,text,true_label,predicted_label,error_type
21,Btw di kantor gue ada anak magang dari kampus pemerintahan basis tni. Tapi hari ini mereka izin masuk 1/2 hari karna mau ada acara bukber sm dekan nya Mau ngerayain abis disahin ruu tni y? abis ini makin bisa banyak rangkap jabatan,0,1,False Positive (FP)
22,udh bukan dwifungsi lg lu pek,0,1,False Positive (FP)
27,Jadi? RUU TNI yg disahkan tuh yg mana ya?,0,1,False Positive (FP)
99,"Institusi ini gak punya integritas, tapi malah mau ditambahin kewenangan oleh DPR. Bikin heboh media dengan beberapa kasus besar seperti kasus Pertamina, tapi cuma nangkep bawahan2nya aja dan kasus nya pun gak jelas gimana...#TolakRUUKejaksaan",0,1,False Positive (FP)
161,"Entar univ univ lain juga diginiin...gabakal ada gerakan demo mahasiswa, udah dipotong sebelum keluar kampus.",0,1,False Positive (FP)
...,...,...,...,...
4873,Dwifungsi Damkar,0,1,False Positive (FP)
4898,Framing supaya demo mahasiswa dinilai ditunggangi dan sebagainya. <url> .,0,1,False Positive (FP)
4899,"Apa sih yang mngkawatirkan dengan demo buruh.buruh mnntut gaji maksimal seperti buruh di negara2 lain di luar negri,blm tnt juga di kabulin DPR,hingga aparat berbuat melebihi batas kek gini.?... Knp GK di temuin aja pimpinan organisasinya di ajak dialog & di arahkan.",0,1,False Positive (FP)
4961,"Keluarga bokap gue hampir semua tni. Dari alm. kakek, om, tante gue hampir semua-muanya tentara. Semalem liat postingan sepupu dia bilang si om jg menolak ruu tni. Alhamdulillah keluarga gue masih waras.",0,1,False Positive (FP)



--- 🔵 FALSE NEGATIVES (Model said 'Not Relevant', but was 'Relevant') ---


,text,true_label,predicted_label,error_type
10,Masa depan tak pasti. TNI harus pasti kuat. UU TNI adalah bekal menuju ke sana. #TNIKuatIndonesiaAman,1,0,False Negative (FN)
44,awas bangkitnya #DwiFungsiABRI bangkit kembali <url>,1,0,False Negative (FN)
74,"Mahasiswa Demo di Balai Kota Solo, Balas Aksi Unjuk Rasa yang Ditemui Gibran <url>",1,0,False Negative (FN)
76,Demonstrasi mcmni penting sebenarnya untuk ajar ke budak-budak dan orang dewasa. Sebab kalau setakat terang secara teori tapi tak tunjuk situasi sebenar org mudah lupa apa yang sepatutnya buat tambah-tambah bila dah kalut dan panik. Cumanya kenapa tak padam-padam abang bomba ui...,1,0,False Negative (FN)
81,"Teror Terhadap Jurnalis, Teror Terhadap Papua Koalisi Advokasi Keadilan dan Keselamatan Jurnalis di Tanah Papua yang terdiri dari para jurnalis dan pembela HAM menggelar demonstrasi damai di depan Mapolda Papua, di Kota Jayapura, Papua pada Selasa, 17 Desember 2024.",1,0,False Negative (FN)
...,...,...,...,...
4949,"In syaa Allah, butuh perubahan yang mendasar mulai dari pemimpinnya... Tidak dalam waktu dekat, kita berharap pencalonan Anies Baswedan menjadi pintu masuk perubahan yang mendasar ini. Namun upaya penjegalan akan terus diusahakan oleh mereka. Jangan biarkan #IndonesiaGelap",1,0,False Negative (FN)
4951,Aksi masyarakat Kota Sorong menyerukan dukungan terhadap Undang-Undang (UU) TNI melalui aksi sosial pembagian takjil yang digelar pada. Minggu (23/3/2025). <url> #TNIADbekerjadenganhati #TNIADDihatiRakyat #TNIADPrima #TNIADBerjuangBersamaRakyat,1,0,False Negative (FN)
4963,"Mana mungkin dipublikasi dengan menunjukan ijazahnya,dan mana mungkin kepolisian melawan junjungannya,salam #indonesiagelap",1,0,False Negative (FN)
4968,Ini unjuk rasa masalah apa lagi ya?,1,0,False Negative (FN)


In [42]:
def calculate_weighted_metrics(eval_metrics):
    """
    Calculates the weighted-average F1, precision, and recall
    from a Hugging Face eval metrics dictionary.
    """
    
    # --- Get F1 and Support ---
    f1_false = eval_metrics.get('eval_False_f1')
    support_false = eval_metrics.get('eval_False_support')
    f1_true = eval_metrics.get('eval_True_f1')
    support_true = eval_metrics.get('eval_True_support')
    
    # --- Get Precision ---
    precision_false = eval_metrics.get('eval_False_precision')
    precision_true = eval_metrics.get('eval_True_precision')
    
    # --- Get Recall ---
    recall_false = eval_metrics.get('eval_False_recall')
    recall_true = eval_metrics.get('eval_True_recall')

    # Check that we have the minimum required keys
    if None in [f1_false, support_false, f1_true, support_true]:
        print("Error: Missing required keys for F1/support.")
        return {}

    # --- Calculate Total Support ---
    total_support = support_false + support_true
    if total_support == 0:
        print("Error: Total support is zero.")
        return {}

    # --- Calculate Weighted Averages ---
    weighted_f1 = ( (f1_false * support_false) + (f1_true * support_true) ) / total_support
    
    weighted_precision = ( (precision_false * support_false) + (precision_true * support_true) ) / total_support
    
    weighted_recall = ( (recall_false * support_false) + (recall_true * support_true) ) / total_support

    return {
        "weighted_f1": weighted_f1,
        "weighted_precision": weighted_precision,
        "weighted_recall": weighted_recall,
        "total_support": total_support
    }

In [43]:
print(calculate_weighted_metrics(results))
print(results)

{'weighted_f1': 0.9085421117438722, 'weighted_precision': 0.9094002209137451, 'weighted_recall': 0.9082, 'total_support': 5000}
{'eval_loss': 0.24182546138763428, 'eval_accuracy': 0.9082, 'eval_macro_f1': 0.9037855219555779, 'eval_macro_precision': 0.9010565164892763, 'eval_macro_recall': 0.9070816456317661, 'eval_False_precision': 0.8649052841475573, 'eval_False_recall': 0.9022360894435777, 'eval_False_f1': 0.8831763807584627, 'eval_False_support': 1923, 'eval_True_precision': 0.9372077488309953, 'eval_True_recall': 0.9119272018199545, 'eval_True_f1': 0.9243946631526931, 'eval_True_support': 3077, 'eval_runtime': 2.0899, 'eval_samples_per_second': 2392.416, 'eval_steps_per_second': 9.57, 'epoch': 10.0}


In [45]:
import re, unicodedata, jaconv, emoji

_URL      = re.compile(r'https?://\S+')
_MENTION  = re.compile(r'@\w+')
_WS       = re.compile(r'\s+')
_KUTI_CUT = re.compile(r'(?i)kutipan.*$', re.DOTALL)

# --- (MODIFIED) ---
# Catches "word" + "dari" + "domain.com" -> replaces with "word"
# Changed \w+ to \S+ to include punctuation like '!'
_DARI_URL_ATTACHED = re.compile(r'(\S+)dari\s+([a-z0-9.-]+\.[a-z]{2,})\b', re.I)

# Catches " dari " + "domain.com" -> replaces with empty string
_DARI_URL_SPACED = re.compile(r'\s+dari\s+([a-z0-9.-]+\.[a-z]{2,})\b', re.I)

# --- (NEW) ---
# Catches any word ending in "dari" (e.g., "anarko!dari", "negaradari")
_DARI_STUCK = re.compile(r'(\S+)dari\b', re.I)

def cleantext(row: str):
    text = row["content"] #type: ignore
    text = unicodedata.normalize('NFKC', text)
    text = jaconv.z2h(text, kana=False, digit=True, ascii=True)
    text = text.replace("tanya grok", " ")
    text = text.replace("grokproductivitypasang", " ")
    text = text.replace('\\n', ' ').replace('\\r', ' ')

    # Handle standard URLs first
    text = _URL.sub(' <url> ', text)
    text = text.replace('ini tidak tersedia', ' ')

    text = _MENTION.sub('@USER', text)
    text = re.sub(r'^rt\s+', '', text, flags=re.I)
    text = re.sub(r'(\b\d{4})(?=[a-zA-Z])', r'\1 ', text)
    text = _KUTI_CUT.sub('', text)

    # text = _DARI_URL_ATTACHED.sub(r'\1', text)
    # text = _DARI_URL_SPACED.sub('', text)
    # text = _DARI_STUCK.sub(r'\1', text)

    text = _WS.sub(' ', text).strip()
    row["content"] = text #type: ignore
    return row

In [55]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from scipy.special import softmax # Or use torch.nn.functional.softmax
import numpy as np
import pandas as pd
from tqdm import tqdm
trained_model.eval()     # Put model in evaluation mode (turns off dropout)

# 3. Prepare your new data
dataset = load_dataset("tianharjuno/twitter-parse", cache_dir="/data/cache")
raw_ds = dataset["source_stage_2"]
raw_ds = raw_ds.map(cleantext, num_proc=30)

new_texts = raw_ds["content"]

batch_size = 256  # <-- Adjust this based on your GPU memory. Try 16, 32, or 64.

# --- 5. NEW: Batch Prediction Loop ---
all_probabilities = []
all_predicted_indices = []

print(f"Starting prediction on {len(new_texts)} texts in batches of {batch_size}...")

# Use torch.no_grad() for the whole loop
with torch.no_grad():
    # Loop over the new_texts in chunks
    for i in tqdm(range(0, len(new_texts), batch_size)):
        
        # Get the current batch of texts
        batch_texts = new_texts[i : i + batch_size]
        
        # Tokenize the batch
        inputs = tokenizer(
            batch_texts, 
            padding=True,       # Pad to the longest text *in this batch*
            truncation=True,    # Truncate to model's max length
            max_length=512,     # Explicitly set max length
            return_tensors="pt"
        )
        
        # Move batch to GPU
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Run inference
        outputs = trained_model(**inputs)
        
        # Get logits and convert to probabilities
        logits = outputs.logits
        probabilities = softmax(logits.cpu().numpy(), axis=1)
        
        # Get predicted indices
        predicted_indices = np.argmax(probabilities, axis=1)
        
        # Store results
        all_probabilities.append(probabilities)
        all_predicted_indices.append(predicted_indices)

print("Prediction complete.")

# --- 6. Consolidate Results ---
# Combine the results from all batches
all_probabilities = np.concatenate(all_probabilities, axis=0)
all_predicted_indices = np.concatenate(all_predicted_indices, axis=0)

# --- 7. Package into DataFrame ---
results_data = []
for i in range(len(new_texts)):
    results_data.append({
        'text': new_texts[i],
        'predicted_label': class_labels.names[all_predicted_indices[i]],
        'confidence': all_probabilities[i].max(),
        'prob_False': all_probabilities[i][0],
        'prob_True': all_probabilities[i][1]
    })

df = pd.DataFrame(results_data)
print(df)

README.md: 0.00B [00:00, ?B/s]

data/source_labeled-00000-of-00001.parqu(…):   0%|          | 0.00/32.3M [00:00<?, ?B/s]

Generating source_stage_1 split:   0%|          | 0/201583 [00:00<?, ? examples/s]

Generating source_stage_2 split:   0%|          | 0/247820 [00:00<?, ? examples/s]

Generating cleaned split:   0%|          | 0/195952 [00:00<?, ? examples/s]

...

Generating source_labeled split:   0%|          | 0/247820 [00:00<?, ? examples/s]

Map (num_proc=30):   0%|          | 0/247820 [00:00<?, ? examples/s]

Starting prediction on 247820 texts in batches of 256...


100%|██████████| 969/969 [01:56<00:00,  8.33it/s]


Prediction complete.
                                                                                                                                                                                                                                                                                     text  \
0                                                                                                                                                                                                                                                Kembalikan Tentara ke barak #TolakRUUTNI   
1                                                                                                                                                                                                                                           Masih ga nyangka suamiku tentara #tolakruutni   
2                                                                                                                           

In [57]:
source_ds = ds["source_stage_2"]
predictions_list = df["predicted_label"].tolist()
# 3. Define the function that .map() will run on each row
def update_row(example, idx):
    """
    Takes a single row (example) and its index (idx).
    It replaces the value in 'relevant' with the new prediction.
    """
    example['relevant'] = predictions_list[idx]
    return example

# 4. Apply the function to the entire dataset
# .map() returns a new dataset, so we must re-assign it
print("Starting to update 'relevant' column...")
source_ds = source_ds.map(update_row, with_indices=True)

print("Column updated successfully!")
print(source_ds)

# Check the first row to see the change
# You should now see your string label (e.g., "True" or "False")
# instead of the old boolean (e.g., True or False)
print(source_ds[5])

Starting to update 'relevant' column...
Column updated successfully!
Dataset({
    features: ['tweet_id', 'time', 'author', 'content', 'comment_count', 'repost_count', 'like_count', 'view_count', 'relevant', 'sentiment'],
    num_rows: 247820
})
{'tweet_id': '1909950840775020808', 'time': '2025-04-09T12:45:40.000', 'author': '@lightfuryy_', 'content': 'Presiden kita ini udah KOSONG dari pas zaman debat pilpres. Dan skrg negara kita di pimpin oleh 2 manusia KOSONG bin DONGO \n\n#TolakRUUTNI #SupremasiSipil', 'comment_count': 0, 'repost_count': 2, 'like_count': 1, 'view_count': 463, 'relevant': True, 'sentiment': 0}


In [58]:
view_ds = source_ds.select_columns(["content", "relevant"])
for row in view_ds.to_list()[:10]:
    print(row)

{'content': 'Kembalikan Tentara ke barak\n#TolakRUUTNI', 'relevant': True}
{'content': 'Masih ga nyangka suamiku tentara #tolakruutni', 'relevant': True}
{'content': 'Beberapa zine yg akan saya bawa saat melapak nanti\n\n#TolakRUUTNI \n#SupremasiSipil', 'relevant': True}
{'content': 'Sebagai mahasiswa hukum, kami memiliki tanggung jawab akademik untuk mengawal proses pembentukan undang-undang agar memiliki proses partisipasi yg bermakna!\n\nberjuang turun ke jalan, bahkan berjuang di ruang sidang, semua sah dilakukan. yg penting #tolakruutni', 'relevant': True}
{'content': 'sekali-kali ga nyebelin ye kite #FallingInLoveEra #TolakRUUTNI', 'relevant': True}
{'content': 'Presiden kita ini udah KOSONG dari pas zaman debat pilpres. Dan skrg negara kita di pimpin oleh 2 manusia KOSONG bin DONGO \n\n#TolakRUUTNI #SupremasiSipil', 'relevant': True}
{'content': 'Kalo bisa menteri dan pejabat negara lainnya ikut disandera juga.. wkwkwkkwk\n\n#TolakRUUTNI QRIS', 'relevant': True}
{'content': 'Gre

In [59]:
ds["source_labeled"] = source_ds

In [60]:
ds.push_to_hub("tianharjuno/twitter-parse", commit_message="Labeled the source ds")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/202 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/248 [00:00<?, ?ba/s]

...

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/248 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/tianharjuno/twitter-parse/commit/72c6aa86902d8e64ae2f44c3192c13817609d206', commit_message='Labeled the source ds', commit_description='', oid='72c6aa86902d8e64ae2f44c3192c13817609d206', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/tianharjuno/twitter-parse', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tianharjuno/twitter-parse'), pr_revision=None, pr_num=None)

In [61]:
trained_model.push_to_hub("tianharjuno/ruu-tni-relevancy-classification-p1", commit_description="Changed training pipeline")
tokenizer.push_to_hub("tianharjuno/ruu-tni-relevancy-classification-p1", commit_description="Changed training pipeline")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmp9urvown6/model.safetensors    :   8%|7         | 33.5MB /  442MB            

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/tianharjuno/ruu-tni-relevancy-classification-p1/commit/577a485da72ce07eded842332f974dd60fb7b73a', commit_message='Upload tokenizer', commit_description='Changed training pipeline', oid='577a485da72ce07eded842332f974dd60fb7b73a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tianharjuno/ruu-tni-relevancy-classification-p1', endpoint='https://huggingface.co', repo_type='model', repo_id='tianharjuno/ruu-tni-relevancy-classification-p1'), pr_revision=None, pr_num=None)